# Hello Bluesky: Reading detectors and scanning

In this notebook you will:

* Connect to some simulated hardware.
* Acquire some data via two common experimental procedures ("plans"), ``count`` and ``scan``.
* Write a custom plan.

We will use the following modules---please import them before continuing

In [1]:
import matplotlib
%matplotlib widget
import bluesky
from bluesky import RunEngine
from bluesky.callbacks.best_effort import BestEffortCallback
import yaqc_bluesky

Next we need to create an configure a runengine.

In [2]:
RE = RunEngine()
bec = BestEffortCallback()
RE.subscribe(bec)

0

## Configuration

Run the following cell to ensure that all of the yaq daemons are running properly. Everything should be online and not busy.

In [3]:
%%bash
yaqd status

[Errno 111] Connection refused
+-----------+-------+--------------------------+------+---------+-------+
| host      | port  | kind                     | name | status  | busy  |
+-----------+-------+--------------------------+------+---------+-------+
| 127.0.0.1 | 38765 | fake-continuous-hardware | test | offline | ?     |
| 127.0.0.1 | 38100 | fake-continuous-hardware | d1   | online  | False |
| 127.0.0.1 | 38101 | fake-continuous-hardware | d2   | online  | False |
| 127.0.0.1 | 38102 | fake-continuous-hardware | d3   | online  | False |
| 127.0.0.1 | 38200 | fake-triggered-sensor    | pmt  | online  | False |
+-----------+-------+--------------------------+------+---------+-------+


Now, let's generate yaqc-bluesky device objects for each of our daemons.

In [4]:
m1 = yaqc_bluesky.Device(38100)
m2 = yaqc_bluesky.Device(38101)
m3 = yaqc_bluesky.Device(38102)
s1 = yaqc_bluesky.Device(38200)

## Count

In [5]:
from bluesky.plans import count
RE(count([s1]))



Transient Scan ID: 1     Time: 2020-07-16 17:55:57
Persistent Unique Scan ID: '8440795e-bc58-403a-8bcf-1d5778ff1dee'
New stream: 'primary'
+-----------+------------+-----------------+
|   seq_num |       time | pmt_random_walk |
+-----------+------------+-----------------+
|         1 | 17:55:57.7 |          -0.258 |
+-----------+------------+-----------------+
generator count ['8440795e'] (scan num: 1)





('8440795e-bc58-403a-8bcf-1d5778ff1dee',)

## Scan

In [7]:
from bluesky.plans import scan
RE(scan([s1], m1, -10, 10, 15))



Transient Scan ID: 3     Time: 2020-07-16 17:56:17
Persistent Unique Scan ID: '02f4602b-1e41-4923-b1a7-563d4ea8b768'
New stream: 'primary'
+-----------+------------+-------------+-----------------+
|   seq_num |       time | d1_readback | pmt_random_walk |
+-----------+------------+-------------+-----------------+
|         1 | 17:56:19.0 |       0.000 |          -0.216 |
|         2 | 17:56:19.2 |       0.000 |           0.140 |
|         3 | 17:56:19.2 |       0.000 |          -0.007 |
|         4 | 17:56:19.3 |       0.000 |           0.017 |
|         5 | 17:56:19.4 |       0.000 |          -0.258 |
|         6 | 17:56:19.5 |       0.000 |          -0.186 |
|         7 | 17:56:19.5 |       0.000 |           0.344 |
|         8 | 17:56:19.6 |       0.000 |          -0.138 |
|         9 | 17:56:20.9 |       1.000 |          -0.215 |
|        10 | 17:56:21.0 |       1.000 |          -0.148 |
|        11 | 17:56:21.1 |       1.000 |           0.002 |
|        12 | 17:56:21.2 |       

/home/blaise/miniconda3/envs/yaq/lib/python3.8/site-packages/bluesky/callbacks/fitting.py:284: RuntimeWarning: divide by zero encountered in double_scalars
  m = dy / dx


('02f4602b-1e41-4923-b1a7-563d4ea8b768',)

Bluesky includes utilities to inspecting plans before they are run. You can imagine various reasons you might want to do this. Example:

In [ ]:
# TODO

## Custom Plan